In [13]:
import env
import pandas as pd
from fredapi import Fred
import requests
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings("ignore")

In [2]:
# use libary api key
fred = Fred(api_key=env.fredkey)
# import neccessary data from FRED API
HousingStart = fred.get_series('HOUST')
interestrate=fred.get_series('FEDFUNDS')
unem=fred.get_series('UNRATE')
CPI=fred.get_series('CPIAUCSL')
DGS10=fred.get_series('DGS10')
PPI=fred.get_series('PPIACO')
coreCPI=fred.get_series('CPILFESL')
recession=fred.get_series('SAHMREALTIME')
INDPRO=fred.get_series('INDPRO')
USTRADE=fred.get_series('USTRADE')
moneysup=fred.get_series('M2REAL')
RGDP=fred.get_series('GDPC1')
GDP=fred.get_series('GDP')
T10YFFM=fred.get_series('T10YFFM')
housevac=fred.get_series('RHVRUSQ156N')
renvac=fred.get_series('RRVRUSQ156N')
is_recession=fred.get_series('USREC')
# calculate the percentage change over time
pHousingStart=HousingStart.pct_change()
pCPI=CPI.pct_change()
pPPI=PPI.pct_change()
pcoreCPI=coreCPI.pct_change()
pINDPRO=INDPRO.pct_change()
pUSTRADE=USTRADE.pct_change()
pmoneysup=moneysup.pct_change()
pRGDP=RGDP.pct_change()

In [4]:
eco_df=pd.DataFrame(data=[HousingStart,pHousingStart,interestrate,unem,CPI,pCPI,DGS10,PPI,pPPI,coreCPI,pcoreCPI,recession,INDPRO,pINDPRO,USTRADE,pUSTRADE,moneysup,pmoneysup,RGDP,pRGDP,GDP,T10YFFM,is_recession],index=['HousingStart','pHousingStart','interestrate','unem','CPI','pCPI','DGS10','PPI','pPPI','coreCPI','pcoreCPI','recession','INDPRO','pINDPRO','USTRADE','pUSTRADE','moneysup','pmoneysup','RGDP','pRGDP','GDP','T10YFFM','is_recession']).T

In [5]:
eco_df['quarter'] = pd.PeriodIndex(eco_df.index, freq='q')
eco_df=eco_df.groupby('quarter').agg('mean')
eco_df=eco_df.dropna()
eco_df.is_recession[round(eco_df.is_recession,2)==0.67]=1
eco_df.is_recession[round(eco_df.is_recession,2)==0.33]=1

In [11]:
train_size = int(len(eco_df) * .5)
validate_size = int(len(eco_df) * .2)
test_size = int(len(eco_df) - train_size - validate_size)
validate_end_index = train_size + validate_size

# split into train, validation, test
train = eco_df[: train_size]
validate = eco_df[train_size : validate_end_index]
test = eco_df[validate_end_index :]
# combine training and validation data set for exploration purpose
train_val=pd.concat([train,validate])

In [12]:
train_val

,HousingStart,pHousingStart,interestrate,unem,CPI,pCPI,DGS10,PPI,pPPI,coreCPI,...,pINDPRO,USTRADE,pUSTRADE,moneysup,pmoneysup,RGDP,pRGDP,GDP,T10YFFM,is_recession
quarter,,,,,,,,,,,,,,,,,,,,,
1962Q1,1360.666667,0.021731,2.456667,5.633333,30.106667,0.001774,4.016290,31.700000,0.001055,31.233333,...,0.004359,5625.200000,0.002369,1130.066667,0.005692,3502.298,0.017836,594.013,1.560000,0.0
1962Q2,1470.333333,-0.008013,2.606667,5.533333,30.220000,0.000442,3.875714,31.533333,-0.002106,31.366667,...,-0.000354,5673.933333,0.001648,1149.666667,0.005539,3533.947,0.009037,600.366,1.266667,0.0
1962Q3,1430.333333,-0.016085,2.846667,5.566667,30.306667,0.002313,3.989524,31.700000,0.004223,31.466667,...,0.005671,5686.800000,0.001673,1164.200000,0.003019,3577.362,0.012285,609.027,1.143333,0.0
1962Q4,1573.000000,0.060051,2.923333,5.533333,30.380000,-0.000438,3.902459,31.666667,-0.003141,31.533333,...,0.001749,5710.266667,0.000923,1184.666667,0.007712,3589.128,0.003289,612.280,0.980000,0.0
1963Q1,1411.333333,0.006462,2.966667,5.766667,30.476667,0.001424,3.892131,31.533333,-0.001055,31.600000,...,0.008294,5737.300000,0.002154,1207.233333,0.005857,3628.306,0.010916,621.672,0.926667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003Q1,1736.000000,-0.008329,1.250000,5.866667,183.366667,0.003837,3.920000,138.033333,0.020407,192.466667,...,0.002132,14933.000000,-0.001827,3182.466667,0.001300,13619.434,0.005167,11174.129,2.670000,0.0
2003Q2,1753.666667,0.027964,1.246667,6.133333,183.066667,-0.001450,3.620635,137.166667,-0.007461,192.800000,...,-0.001751,14906.466667,-0.000194,3250.933333,0.009062,13741.107,0.008934,11312.766,2.373333,0.0
2003Q3,1889.666667,0.013387,1.016667,6.133333,184.433333,0.003628,4.227656,138.066667,0.001209,193.566667,...,0.003145,14912.700000,0.001089,3292.300000,0.000639,13970.157,0.016669,11566.669,3.216667,0.0
